In [82]:
import gymnasium as gym
%env MUJOCO_GL=egl
from datasets import load_from_disk, concatenate_datasets


env: MUJOCO_GL=egl


In [72]:
dataset_1 = load_from_disk("BC-MLP-MSE/datasets/grasp_100_2024-09-06_17-03-47.hf")
dataset_2 = load_from_disk("BC-MLP-MSE/datasets/one_trajectory_25_2024-09-06_20-44-13.hf")

d1_eps = range(45)
d2_eps = range(5)

dataset_1_filtered = dataset_1.filter(lambda example: example["episode_index"] in d1_eps)
dataset_2_filtered = dataset_2.filter(lambda example: example["episode_index"] in d2_eps)



In [69]:
d2_eps = [1]
dataset_2_filtered = dataset_2.filter(lambda example: example["episode_index"] in d2_eps)


In [64]:
dataset_2_filtered['episode_index']
dataset_2_filtered['episode_index'][-1]

# 'episode_index', 'index'

1

In [73]:
dataset_1_filtered = dataset_1_filtered.map(lambda example, idx: {"episode_index": d1_eps.index(example["episode_index"]), "index": idx}, with_indices=True)
dataset_2_filtered = dataset_2_filtered.map(lambda example, idx: {"episode_index": d2_eps.index(example["episode_index"]) + dataset_1_filtered["episode_index"][-1] + 1, "index": idx + dataset_1_filtered["index"][-1] + 1}, with_indices=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [80]:
dataset_2_filtered["index"][-1]

9999

In [83]:
dataset = concatenate_datasets([dataset_1_filtered, dataset_2_filtered])      


In [98]:
assert(dataset["index"] == list(range(len(dataset))))
assert(list(set(dataset["episode_index"])) == list(range(len(d1_eps) + len(d2_eps))))

In [ ]:
import datetime
dataset.save_to_disk(f"BC-MLP-MSE/datasets45_diverse_5_single_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.hf")
